In [1]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform
import warnings
warnings.filterwarnings('ignore')

In [55]:
## Paths Input Here
analysis = "roi_roi_overlap/vta_and_bf_ch_nuclei"
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/seeds/non_fl_vtas_bilateral'
    roi_path = r'/Users/cu135/Dropbox (Partners HealthCare)/neurotransmitters/basal_forebrain/generated_spherical_rois/cholinergic_nuclei_maxima'
    
    # clin_path = 'path to clinical values'
    out_dir = os.path.join(os.path.dirname(conn_path), f'{analysis}')
    
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Mac style')
    print('I will save to:', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    print('add the paths')

if os.path.isdir(out_dir) != True:
    os.makedirs(out_dir)

save = True

I have set pathnames in the Mac style
I will save to: /Users/cu135/Dropbox (Partners HealthCare)/memory/seeds/roi_roi_overlap/vta_and_bf_ch_nuclei


In [56]:
#Get and Prepare Dataframes
from calvin_utils.import_matrices import import_matrices_from_folder
df_one = import_matrices_from_folder(conn_path, file_pattern='/*.nii')
display(df_one)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/memory/seeds/non_fl_vtas_bilateral/*.nii
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further s

,1202_vat_seed_compound_fMRI_efield.nii,149_vat_seed_compound_fMRI_efield.nii,137_vat_seed_compound_fMRI_efield.nii,144_vat_seed_compound_fMRI_efield.nii,126_vat_seed_compound_fMRI_efield.nii,112_vat_seed_compound_fMRI_efield.nii,103_vat_seed_compound_fMRI_efield.nii,101_vat_seed_compound_fMRI_efield.nii,110_vat_seed_compound_fMRI_efield.nii,146_vat_seed_compound_fMRI_efield.nii,...,133_vat_seed_compound_fMRI_efield.nii,140_vat_seed_compound_fMRI_efield.nii,122_vat_seed_compound_fMRI_efield.nii,142_vat_seed_compound_fMRI_efield.nii,120_vat_seed_compound_fMRI_efield.nii,131_vat_seed_compound_fMRI_efield.nii,105_vat_seed_compound_fMRI_efield.nii,114_vat_seed_compound_fMRI_efield.nii,108_vat_seed_compound_fMRI_efield.nii,119_vat_seed_compound_fMRI_efield.nii
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
roi_df = import_matrices_from_folder(roi_path, file_pattern='/*.nii')
display(roi_df)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/neurotransmitters/basal_forebrain/generated_spherical_rois/cholinergic_nuclei_maxima/*.nii
cannot further split name
cannot further split name
cannot further split name
cannot further split name


,ch_1_2.nii,ch4p.nii,ch3.nii,ch4.nii
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
902624,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0


In [58]:
# Calculate overlap for each ROI and column
overlap = {}
#Set specific ROI to 1 if nonzero
roi_df[roi_df!= 0] = 1
#Set specific nifti of interest to 1 if nonzero
df_one[df_one!= 0] = 1

#Iterate over ROI niftis
for i in range(0, len(roi_df.columns)):
    #Initialize
    row = []
    
    #Iterate over nifti of comparison
    for j in range(len(df_one.columns)):
        # Find all values where the data frames overlap (both == 1)
        numerator = np.sum(np.logical_and(roi_df.iloc[:, i], df_one.iloc[:, j]))

        # Find total number of nonzero values in df_one
        denominator = np.sum(roi_df.iloc[:, i])

        # Find total percent of overlap
        overlap_percent = numerator / denominator * 100
        row.append(overlap_percent)
    overlap[roi_df.columns[i]] = row


In [67]:
# Create a data frame with the overlap values
overlap_df = pd.DataFrame(overlap, index=df_one.columns)

# overlap_df = pd.DataFrame(overlap, columns=df_one.columns, ).transpose()
display(overlap_df)

#Merge the dataframe back to the orignal
if save:
    overlap_df.to_csv(os.path.join(out_dir, 'overlap_df.csv'))
    print('saved to ', out_dir)

,ch_1_2.nii,ch4p.nii,ch3.nii,ch4.nii
1202_vat_seed_compound_fMRI_efield.nii,0.000000,0.0,12.903226,0.0
149_vat_seed_compound_fMRI_efield.nii,97.058824,0.0,100.000000,0.0
137_vat_seed_compound_fMRI_efield.nii,64.705882,0.0,87.096774,0.0
144_vat_seed_compound_fMRI_efield.nii,14.705882,0.0,32.258065,0.0
126_vat_seed_compound_fMRI_efield.nii,73.529412,0.0,93.548387,0.0
112_vat_seed_compound_fMRI_efield.nii,14.705882,0.0,3.225806,0.0
103_vat_seed_compound_fMRI_efield.nii,8.823529,0.0,16.129032,0.0
101_vat_seed_compound_fMRI_efield.nii,0.000000,0.0,12.903226,0.0
110_vat_seed_compound_fMRI_efield.nii,41.176471,0.0,29.032258,0.0
146_vat_seed_compound_fMRI_efield.nii,35.294118,0.0,51.612903,0.0


saved to  /Users/cu135/Dropbox (Partners HealthCare)/memory/seeds/roi_roi_overlap/vta_and_bf_ch_nuclei
